In [1]:
raw_data_dir = '../data/raw/'
interim_data_dir = '../data/interim'

In [2]:
import requests
import json
import pandas as pd

url= "https://crashviewer.nhtsa.dot.gov/CrashAPI"
#/crashes/GetCrashesByLocation?fromCaseYear=2014&toCaseYear=2015&state=1&county=1&format=json

fromCaseYear = "2010"
toCaseYear = "2020"
state = "6"
qurl = f"{url}/crashes/GetCrashesByLocation?fromCaseYear={fromCaseYear}&toCaseYear={toCaseYear}&state={state}&county=73&format=json"

cali=requests.get(qurl).json()
data = (json.dumps(cali,indent=4,sort_keys=True))

CITYNAME = []
COUNTY =  []
COUNTYNAME = []
CaseYear = []
FATALS = []
LATITUDE = []
LONGITUD = []
STATE = []
STATENAME = []
ST_CASE = []
TOTALVEHICLES = []
TWAY_ID = []
TWAY_ID2 = []
VE_FORMS = []
# Loop through the list of cities and perform a request for data on each
for COUNTYNAME in COUNTYNAME:
    response = requests.get(qurl).json()
    CITYNAME.append(response[0]['CITYNAME'])


In [3]:
cali_df = pd.DataFrame(cali['Results'][0])
cali_df

,CITY,CITYNAME,COUNTY,COUNTYNAME,CaseYear,FATALS,LATITUDE,LONGITUD,STATE,STATENAME,ST_CASE,TOTALVEHICLES,TWAY_ID,TWAY_ID2,VE_FORMS
0,3260,SAN DIEGO,73,SAN DIEGO (73),2014,1,32.76605000,-117.150869440,6,California,60005,2,I-8,,2
1,0,NOT APPLICABLE,73,SAN DIEGO (73),2013,1,33.37609444,-117.257936110,6,California,60006,2,CR-ALTURAS RD,,1
2,3260,SAN DIEGO,73,SAN DIEGO (73),2013,1,32.80097222,-117.142119440,6,California,60007,4,MURRAY RIDGE RD,,1
3,3260,SAN DIEGO,73,SAN DIEGO (73),2018,1,32.57529722,-117.081208330,6,California,60007,1,I-5,,1
4,1080,EL CAJON,73,SAN DIEGO (73),2014,1,32.79522222,-116.935727780,6,California,60008,2,JAMACHA ROAD,,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,710,CHULA VISTA,73,SAN DIEGO (73),2016,1,32.60915556,-117.019969440,6,California,63474,2,OLYMPIC PKWY,,2
1891,1200,ESCONDIDO,73,SAN DIEGO (73),2017,1,33.07225278,-117.059525000,6,California,63518,2,BEAR VALLEY PKWY,,2
1892,1876,LEMON GROVE,73,SAN DIEGO (73),2016,1,32.73855833,-117.042861110,6,California,63569,1,MASSACHUSETTS AVE,,1
1893,710,CHULA VISTA,73,SAN DIEGO (73),2017,1,32.63021944,-117.092161110,6,California,63582,1,H ST,,1


In [4]:
def extract_people(v):
    for p in v['Persons']:
        yield {
            'Speed Limit Exceeded': v['SPEEDRELNAME'],
            'Speed Limit': v['TRAV_SP'],
            'Vin Number': v['VINNAME'],
            'Traveled Speed Veh': v['VSPD_LIM'],
            'Make': v['MAKENAME'],
            'Make/Model': v['MAK_MODNAME'],
            'Model': v['MODELNAME'],
            
            "Age": p['AGE'],
            "Age Name": p['AGENAME'],
            "County": p['COUNTYNAME'],
            "Death DANAME": p['DEATH_DANAME'],
            "DOA Name": p['DOANAME'],
            "Injury": p['INJ_SEVNAME'],
            "Race": p['RACENAME'],
            "Age": p["AGENAME"],
            "Death": p["DOANAME"],
            "Road_Type": p["ROAD_FNCNAME"],
            "Sex": p["SEXNAME"],
            "Make": p["MAKENAME"],
        }


        
def get_people(case):

    hour = case['HOUR']
    minute = case['MINUTE']
    time = f"{hour}:{minute}"
        
    accident_info = {
        'Case Number': case['ST_CASE'],
        "Description of Veh Coll": case['CF2NAME'], 
        "Day of Week": case['DAY_WEEKNAME'],
        "Drunk Driver": case['DRUNK_DR'],
        "Year": case['CaseYear'],
        "Month": case['MonthName'],
        "Hour": hour,
        "Time of Accident": time,
    }

    vehicles = case['Vehicles']
    
    people = [{**accident_info, **p} for v in vehicles for p in extract_people(v)]
    
    return pd.DataFrame(people)

def get_events(case):
    c_events = [{
        'Case Number': case['ST_CASE'],
# In a traffic accident AOI is Area of Impact. The spot the two cars collided is measured 
# to a fixed object, usually the curb, so it can be reconstructed later.
        'Area of Impact': e['AOI1NAME'],
# standard of evidence
# https://safety.fhwa.dot.gov/rsdp/cdip_rpti.aspx
        'Standard of Evenidence': e['SOENAME'],
        'Event Number': e['EVENTNUM'],
        'Vehicle 1': e['VNUMBER1'],
        'Vehicle 2': e['VNUMBER2'],
    } for e in case['CEvents']]
    
    return pd.DataFrame(c_events)




In [5]:
fromCaseYear = "2010"
toCaseYear = "2020"
state = "6"

def get_cases():
    for i, r in cali_df.iterrows():
        statecase = r["ST_CASE"]
        caseyear = r["CaseYear"]
        qurl = f"{url}/crashes/GetCaseDetails?stateCase={statecase}&caseYear={caseyear}&state=6&format=json"
        data = requests.get(qurl).json()
        case = data['Results'][0][0]['CrashResultSet']
        yield case

In [6]:
people_list = [get_people(case) for case in get_cases()]
people_df = pd.concat(people_list)
people_df

,Case Number,Description of Veh Coll,Day of Week,Drunk Driver,Year,Month,Hour,Time of Accident,Speed Limit Exceeded,Speed Limit,...,Age,Age Name,County,Death DANAME,DOA Name,Injury,Race,Death,Road_Type,Sex
0,60005,"Motor Vehicle struck by falling cargo,or somet...",Thursday,0,2014,January,16,16:10,"Yes, Exceeded Speed Limit",95,...,22 Years,22 Years,SAN DIEGO (73),9,Died at Scene,Fatal Injury (K),Black,Died at Scene,Urban-Principal Arterial-Interstate,Male
1,60005,"Motor Vehicle struck by falling cargo,or somet...",Thursday,0,2014,January,16,16:10,No,45,...,53 Years,53 Years,SAN DIEGO (73),Not Applicable (Non-Fatal),Not Applicable,No Apparent Injury (O),Not a Fatality (not Applicable),Not Applicable,Urban-Principal Arterial-Interstate,Female
0,60006,None,Thursday,1,2013,January,18,18:8,No,30,...,67 Years,67 Years,SAN DIEGO (73),3,Died at Scene,Fatal Injury (K),White,Died at Scene,Rural-Local Road or Street,Male
0,60007,None,Tuesday,1,2013,January,0,0:15,"Yes, Too Fast for Conditions",73,...,21 Years,21 Years,SAN DIEGO (73),19,Not Applicable,Fatal Injury (K),White,Not Applicable,Urban-Minor Arterial,Female
0,60007,None,Thursday,0,2018,January,17,17:25,No,75,...,30 Years,30 Years,SAN DIEGO (73),Not Applicable (Non-Fatal),Not Applicable,No Apparent Injury (O),Not a Fatality (not Applicable),Not Applicable,None,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,63518,None,Saturday,0,2017,September,10,10:52,No,998,...,32 Years,32 Years,SAN DIEGO (73),Not Applicable (Non-Fatal),Not Applicable,No Apparent Injury (O),Not a Fatality (not Applicable),Not Applicable,None,Male
1,63518,None,Saturday,0,2017,September,10,10:52,No,998,...,32 Years,32 Years,SAN DIEGO (73),10,Not Applicable,Fatal Injury (K),White,Not Applicable,None,Male
0,63569,None,Wednesday,0,2016,July,22,22:15,No,998,...,61 Years,61 Years,SAN DIEGO (73),Not Applicable (Non-Fatal),Not Applicable,No Apparent Injury (O),Not a Fatality (not Applicable),Not Applicable,None,Male
0,63582,None,Thursday,0,2017,November,16,16:51,"Yes, Too Fast for Conditions",999,...,33 Years,33 Years,SAN DIEGO (73),Not Applicable (Non-Fatal),Not Applicable,No Apparent Injury (O),Not a Fatality (not Applicable),Not Applicable,None,Male


In [7]:
event_list = [get_events(case) for case in get_cases()]
event_df = pd.concat(event_list)
event_df

,Case Number,Area of Impact,Standard of Evenidence,Event Number,Vehicle 1,Vehicle 2
0,60005,Non-Collision,Rollover/Overturn,1,1,9999
1,60005,Other Objects Set-In-Motion,Motor Vehicle In-Transport Strikes or is Struc...,2,1,2
2,60005,Not Reported,Motor Vehicle In-Transport,3,1,2
3,60005,Non-Collision,Fire/Explosion,4,1,9999
4,60005,Non-Collision,Fire/Explosion,5,2,9999
...,...,...,...,...,...,...
0,63518,12 Clock Point,Motor Vehicle In-Transport,1,2,1
0,63569,12 Clock Point,Pedestrian,1,1,9999
0,63582,12 Clock Point,Pedestrian,1,1,9999
0,63617,Non-Harmful Event,Ran off Roadway - Direction Unknown,1,1,9999


In [8]:
people_df.to_csv("../Data/people.csv")

In [9]:
event_df.to_csv("../Data/events.csv")